# Batch Optimization (Additive)

Run multiple optimizer/metric jobs in a batch for additive resynthesis using `py_scripts/additive_synth_opt.py`.
Outputs use the same naming scheme (audio, TSV, plots) as in the single-run additive notebook and FM batch notebook.

In [2]:
# Setup and target loading
import os
from pathlib import Path
import numpy as np
import pandas as pd
from IPython.display import display

import matplotlib.pyplot as plt

from py_scripts.additive_synth_opt import (
    AdditiveObjective, additive_synth,
    run_de_optimization, run_dual_annealing, run_basinhopping,
    save_and_display_final_values_additive, make_fft
)
from py_scripts.waveform_generators import sine_wave, square_wave, triangle_wave, sawtooth_wave, noise_wave
from py_scripts.fm_synth_opt import save_wav

# Unified config
SR = 44100
DURATION = 2.0
FFT_PAD = 2
FADE_IN_MS = 10.0
FADE_OUT_MS = 10.0

# Additive-specific
N_PARTIALS = 4  # number of oscillators/partials
USE_WAVEFORMS = True
waveforms = [sine_wave, square_wave, triangle_wave, sawtooth_wave, noise_wave]

# Load target partials (TSV)
target_path = 'tsv/cello_single.tsv'
df = pd.read_csv(target_path, sep='\t')
display(df)
target_freqs = df['Frequency (Hz)'].to_numpy()
# Normalize amplitudes to max=1 for stable objectives
target_amps = (df['Amplitude'] / df['Amplitude'].max()).to_numpy()
target_name = Path(target_path).stem

# Helper: run one additive optimization (delegates to py_scripts)
from py_scripts.additive_batch import run_one_additive as _run_additive_core, RunAdditiveConfig

def run_one_additive(
    *,
    method: str,
    metric: str,
    optimizer_kwargs: dict = None,
    sr: int = SR,
    duration: float = DURATION,
    fft_pad: int = FFT_PAD,
    fade_in_ms: float = FADE_IN_MS,
    fade_out_ms: float = FADE_OUT_MS,
    seed: int | None = 42,
):
    cfg = RunAdditiveConfig(
        target_freqs=target_freqs,
        target_amps=target_amps,
        target_name=target_name,
        n_partials=N_PARTIALS,
        use_waveforms=USE_WAVEFORMS,
        waveforms=tuple(waveforms),
        sr=sr,
        duration=duration,
        fft_pad=fft_pad,
        fade_in_ms=fade_in_ms,
        fade_out_ms=fade_out_ms,
        method=method,
        metric=metric,
        optimizer_kwargs=(optimizer_kwargs or {}),
        seed=seed,
    )
    return _run_additive_core(cfg)


,Frequency (Hz),Amplitude
0,73.684211,0.005356
1,136.842105,0.004395
2,147.368421,0.102953
3,223.684211,0.011307
4,297.368421,0.035448
5,371.052632,0.005977
6,442.105263,0.002880
7,447.368421,0.006474
8,521.052632,0.011020
9,594.736842,0.002818


## Define batch jobs
- `method`: one of `'de'` (Differential Evolution), `'da'` (Dual Annealing), `'bh'` (Basin Hopping).
- `metric`: any supported metric: `'pearson'`, `'mfcc'`, `'itakura_saito'`, `'spectral_convergence'`, `'cosine'`, `'euclidean'`, `'manhattan'`, `'kl'`.
- `kwargs`: optimizer-specific settings (e.g., `maxiter`, `workers`, `stepsize`).

In [3]:
jobs = [
    # Pearson
    dict(method='de', metric='pearson', kwargs=dict(maxiter=500, popsize=20, workers=-1)),
    dict(method='da', metric='pearson', kwargs=dict(maxiter=500)),
    dict(method='bh', metric='pearson', kwargs=dict(maxiter=500, stepsize=0.5)),
    # MFCC
    dict(method='de', metric='mfcc', kwargs=dict(maxiter=500, popsize=20, workers=-1)),
    dict(method='da', metric='mfcc', kwargs=dict(maxiter=500)),
    dict(method='bh', metric='mfcc', kwargs=dict(maxiter=500, stepsize=0.5)),
    # Spectral Convergence
    dict(method='de', metric='itakura_saito', kwargs=dict(maxiter=500, popsize=20, workers=-1)),
    dict(method='da', metric='itakura_saito', kwargs=dict(maxiter=500)),
    dict(method='bh', metric='itakura_saito', kwargs=dict(maxiter=500, stepsize=0.5)),
    # Cosine
    dict(method='de', metric='spectral_convergence', kwargs=dict(maxiter=500, popsize=20, workers=-1)),
    dict(method='da', metric='spectral_convergence', kwargs=dict(maxiter=500)),
    dict(method='bh', metric='spectral_convergence', kwargs=dict(maxiter=500, stepsize=0.5)),

    dict(method='de', metric='cosine', kwargs=dict(maxiter=500, popsize=20, workers=-1)),
    dict(method='da', metric='cosine', kwargs=dict(maxiter=500)),
    dict(method='bh', metric='cosine', kwargs=dict(maxiter=500, stepsize=0.5)),
    # Euclidean
    dict(method='de', metric='euclidean', kwargs=dict(maxiter=500, popsize=20, workers=-1)),
    dict(method='da', metric='euclidean', kwargs=dict(maxiter=500)),
    dict(method='bh', metric='euclidean', kwargs=dict(maxiter=500, stepsize=0.5)),
    # Manhattan
    dict(method='de', metric='manhattan', kwargs=dict(maxiter=500, popsize=20, workers=-1)),
    dict(method='da', metric='manhattan', kwargs=dict(maxiter=500)),
    dict(method='bh', metric='manhattan', kwargs=dict(maxiter=500, stepsize=0.5)),
    # KL
    dict(method='de', metric='kl', kwargs=dict(maxiter=500, popsize=20, workers=-1)),
    dict(method='da', metric='kl', kwargs=dict(maxiter=500)),
    dict(method='bh', metric='kl', kwargs=dict(maxiter=500, stepsize=0.5))
]

rows = []
for j in jobs:
    info = run_one_additive(method=j['method'], metric=j['metric'], optimizer_kwargs=j.get('kwargs') or {})
    rows.append(info)

summary = pd.DataFrame([{
    'method': r['method'],
    'metric': r['metric'],
    'best': r['best'],
    'wav_path': r['wav_path'],
    'tsv_path': r['tsv_path'],
} for r in rows])
display(summary)


Additive 4-osc DE → Pearson:   0%|           0/500 [ETA: ?, Elapsed: 00:00]

[save_wav] wrote: c:\Users\egorp\Nextcloud\code\public_repos\FFTimbre\rendered_audio\optimized_output_additive_cello_single_de_pearson_20250911-015909.wav
TSV file was saved at: c:\Users\egorp\Nextcloud\code\public_repos\FFTimbre\tsv\final_values_additive_cello_single_de_pearson_20250911-015909.tsv
[save_time_plot] wrote: c:\Users\egorp\Nextcloud\code\public_repos\FFTimbre\rendered_plots\optimized_output_additive_cello_single_de_pearson_20250911-015909_time.png
[save_spectrum_plot] wrote: c:\Users\egorp\Nextcloud\code\public_repos\FFTimbre\rendered_plots\optimized_output_additive_cello_single_de_pearson_20250911-015909_spectrum.png
[save_error_plot] wrote: c:\Users\egorp\Nextcloud\code\public_repos\FFTimbre\rendered_plots\optimized_output_additive_cello_single_de_pearson_20250911-015909_error.png


Additive 4-osc DA → Pearson:   0%|           0/500 [ETA: ?, Elapsed: 00:00]

[save_wav] wrote: c:\Users\egorp\Nextcloud\code\public_repos\FFTimbre\rendered_audio\optimized_output_additive_cello_single_da_pearson_20250911-020147.wav
TSV file was saved at: c:\Users\egorp\Nextcloud\code\public_repos\FFTimbre\tsv\final_values_additive_cello_single_da_pearson_20250911-020147.tsv
[save_time_plot] wrote: c:\Users\egorp\Nextcloud\code\public_repos\FFTimbre\rendered_plots\optimized_output_additive_cello_single_da_pearson_20250911-020147_time.png
[save_spectrum_plot] wrote: c:\Users\egorp\Nextcloud\code\public_repos\FFTimbre\rendered_plots\optimized_output_additive_cello_single_da_pearson_20250911-020147_spectrum.png
[save_error_plot] wrote: c:\Users\egorp\Nextcloud\code\public_repos\FFTimbre\rendered_plots\optimized_output_additive_cello_single_da_pearson_20250911-020147_error.png


Additive 4-osc BH → Pearson:   0%|           0/500 [ETA: ?, Elapsed: 00:00]

[save_wav] wrote: c:\Users\egorp\Nextcloud\code\public_repos\FFTimbre\rendered_audio\optimized_output_additive_cello_single_bh_pearson_20250911-021541.wav
TSV file was saved at: c:\Users\egorp\Nextcloud\code\public_repos\FFTimbre\tsv\final_values_additive_cello_single_bh_pearson_20250911-021541.tsv
[save_time_plot] wrote: c:\Users\egorp\Nextcloud\code\public_repos\FFTimbre\rendered_plots\optimized_output_additive_cello_single_bh_pearson_20250911-021541_time.png
[save_spectrum_plot] wrote: c:\Users\egorp\Nextcloud\code\public_repos\FFTimbre\rendered_plots\optimized_output_additive_cello_single_bh_pearson_20250911-021541_spectrum.png
[save_error_plot] wrote: c:\Users\egorp\Nextcloud\code\public_repos\FFTimbre\rendered_plots\optimized_output_additive_cello_single_bh_pearson_20250911-021541_error.png


Additive 4-osc DE → Mfcc:   0%|           0/500 [ETA: ?, Elapsed: 00:00]

[save_wav] wrote: c:\Users\egorp\Nextcloud\code\public_repos\FFTimbre\rendered_audio\optimized_output_additive_cello_single_de_mfcc_20250911-022628.wav
TSV file was saved at: c:\Users\egorp\Nextcloud\code\public_repos\FFTimbre\tsv\final_values_additive_cello_single_de_mfcc_20250911-022628.tsv
[save_time_plot] wrote: c:\Users\egorp\Nextcloud\code\public_repos\FFTimbre\rendered_plots\optimized_output_additive_cello_single_de_mfcc_20250911-022628_time.png
[save_spectrum_plot] wrote: c:\Users\egorp\Nextcloud\code\public_repos\FFTimbre\rendered_plots\optimized_output_additive_cello_single_de_mfcc_20250911-022628_spectrum.png
[save_error_plot] wrote: c:\Users\egorp\Nextcloud\code\public_repos\FFTimbre\rendered_plots\optimized_output_additive_cello_single_de_mfcc_20250911-022628_error.png


Additive 4-osc DA → Mfcc:   0%|           0/500 [ETA: ?, Elapsed: 00:00]

KeyboardInterrupt: 